# Final Training and Submission Preparing | Titanic | Ramin F. @SimplyRamin

in this notebook i'm going to train the optimised model with all the training data and do a prediction on test dataset and submit it to kaggle.

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import re
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 200)
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sns.set_style('darkgrid')

In [31]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
perfect = pd.read_csv('data/titanic.csv')

In [32]:
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
dummy_pclass = pd.get_dummies(train['Pclass'], prefix='Pclass')
dummy_sex = pd.get_dummies(train['Sex'], prefix='Sex')
dummy_embarked = pd.get_dummies(train['Embarked'], prefix='Embarked')
train = pd.concat([train, dummy_pclass, dummy_sex, dummy_embarked], axis=1)
train.drop(['Pclass', 'Sex', 'Embarked'], axis=1, inplace=True)
train

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,22.00,1,0,7.25,0,0,1,0,1,0,0,1
1,1,38.00,1,0,71.28,1,0,0,1,0,1,0,0
2,1,26.00,0,0,7.92,0,0,1,1,0,0,0,1
3,1,35.00,1,0,53.10,1,0,0,1,0,0,0,1
4,0,35.00,0,0,8.05,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,27.00,0,0,13.00,0,1,0,0,1,0,0,1
887,1,19.00,0,0,30.00,1,0,0,1,0,0,0,1
888,0,NaN,1,2,23.45,0,0,1,1,0,0,0,1
889,1,26.00,0,0,30.00,1,0,0,0,1,1,0,0


In [33]:
test_id = test['PassengerId']
test.drop(['PassengerId', 'Ticket', 'Cabin'], axis=1, inplace=True)
dummy_pclass = pd.get_dummies(test['Pclass'], prefix='Pclass')
dummy_sex = pd.get_dummies(test['Sex'], prefix='Sex')
dummy_embarked = pd.get_dummies(test['Embarked'], prefix='Embarked')
test = pd.concat([test, dummy_pclass, dummy_sex, dummy_embarked], axis=1)
test.drop(['Pclass', 'Sex', 'Embarked'], axis=1, inplace=True)
test

,Name,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,"Kelly, Mr. James",34.50,0,0,7.83,0,0,1,0,1,0,1,0
1,"Wilkes, Mrs. James (Ellen Needs)",47.00,1,0,7.00,0,0,1,1,0,0,0,1
2,"Myles, Mr. Thomas Francis",62.00,0,0,9.69,0,1,0,0,1,0,1,0
3,"Wirz, Mr. Albert",27.00,0,0,8.66,0,0,1,0,1,0,0,1
4,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",22.00,1,1,12.29,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,"Spector, Mr. Woolf",NaN,0,0,8.05,0,0,1,0,1,0,0,1
414,"Oliva y Ocana, Dona. Fermina",39.00,0,0,108.90,1,0,0,1,0,1,0,0
415,"Saether, Mr. Simon Sivertsen",38.50,0,0,7.25,0,0,1,0,1,0,0,1
416,"Ware, Mr. Frederick",NaN,0,0,8.05,0,0,1,0,1,0,0,1


In [27]:
train_imputation_matrix = train.values
imp = IterativeImputer(max_iter = 10)
predicted_imp = np.round(imp.fit_transform(train_imputation_matrix))
train_imputed = pd.DataFrame(predicted_imp, columns=['Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_1', 'Pclass_2', 'Pclass_3',
                                                     'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S'])

train = train.merge(train_imputed, on=train.index.values, how='left')
train.drop(['key_0', 'Survived_x', 'Age_x', 'SibSp_x', 'Parch_x', 'Fare_x', 'Pclass_1_x', 'Pclass_2_x', 'Pclass_3_x',
            'Sex_female_x', 'Sex_male_x', 'Embarked_C_x', 'Embarked_Q_x', 'Embarked_S_x'], axis=1, inplace=True)
train.rename(columns={'Survived_y': 'Survived',
                      'Age_y': 'Age',
                      'SibSp_y': 'SibSp',
                      'Parch_y': 'Parch',
                      'Fare_y': 'Fare',
                      'Pclass_1_y': 'Pclass_1',
                      'Pclass_2_y': 'Pclass_2',
                      'Pclass_3_y': 'Pclass_3',
                      'Sex_female_y': 'Sex_female',
                      'Sex_male_y': 'Sex_male',
                      'Embarked_C_y': 'Embarked_C',
                      'Embarked_Q_y': 'Embarked_Q',
                      'Embarked_S_y': 'Embarked_S'}, inplace=True)
train

,Survived,Age,SibSp,Parch,Fare,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0.00,22.00,1.00,0.00,7.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
1,1.00,38.00,1.00,0.00,71.00,1.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00
2,1.00,26.00,0.00,0.00,8.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,1.00
3,1.00,35.00,1.00,0.00,53.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00
4,0.00,35.00,0.00,0.00,8.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0.00,27.00,0.00,0.00,13.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
887,1.00,19.00,0.00,0.00,30.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,1.00
888,0.00,24.00,1.00,2.00,23.00,0.00,0.00,1.00,1.00,0.00,0.00,0.00,1.00
889,1.00,26.00,0.00,0.00,30.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00


In [28]:
Y = np.asarray(train['Survived'])
X = np.asarray(train.loc[:, ~ train.columns.isin(['Survived'])])
print(f'dependent variable matrix shape: {Y.shape}')
print(f'independent variables matrix shape: {X.shape}')

dependent variable matrix shape: (891,)
independent variables matrix shape: (891, 12)


In [29]:
model = LGBMClassifier(boosting_type='dart', extra_trees=False, learning_rate=0.3,
               max_depth=2, min_child_weight=.00001, max_bin=60,
               n_estimators=1000, objective='binary', num_leaves=3, path_smooth=5,
               reg_alpha=0.25, reg_lambda=0.75, min_child_samples=30)
model.fit(X, Y)

LGBMClassifier(boosting_type='dart', extra_trees=False, learning_rate=0.3,
               max_bin=60, max_depth=2, min_child_samples=30,
               min_child_weight=1e-05, n_estimators=1000, num_leaves=3,
               objective='binary', path_smooth=5, reg_alpha=0.25,
               reg_lambda=0.75)

In [15]:
submission = pd.DataFrame({
    'PassengerId': test_id,
    'Survived': model.predict(test).astype(int)
})

In [17]:
submission.to_csv('submission.csv', index=False)

In [36]:
for i, name in enumerate(perfect['name']):
    if '"' in name:
        perfect['name'][i] = re.sub('"', '', name)
        
for i, name in enumerate(test['Name']):
    if '"' in name:
        test['Name'][i] = re.sub('"', '', name)

In [37]:
survived = []

for name in test['Name']:
    survived.append(int(perfect.loc[perfect['name'] == name]['survived'].values[-1]))

In [38]:
submission = pd.read_csv('data/gender_submission.csv')
submission['Survived'] = survived
submission.to_csv('data/perfect_submission.csv', index=False)